In [17]:
DEBUG = False # set this flag to True to use a small subset of data for testing

In [6]:
import wandb
from fastai.vision.all import *

In [12]:
WANDB_PROJECT = "eda"
ENTITY = None # set this to team name if working in a team
BDD_CLASSES = {i:c for i,c in enumerate(['background', 'road', 'traffic light', 'traffic sign', 'person', 'vehicle', 'bicycle'])}
RAW_DATA_AT = 'bdd_simple_1k'
PROCESSED_DATA_AT = 'bdd_simple_1k_split'

In [8]:

URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'
path = Path(untar_data(URL, force_download=True))

In [9]:
path.ls()

(#3) [Path('C:/Users/ShubhaManikarnike/.fastai/data/bdd_simple_1k/images'),Path('C:/Users/ShubhaManikarnike/.fastai/data/bdd_simple_1k/labels'),Path('C:/Users/ShubhaManikarnike/.fastai/data/bdd_simple_1k/LICENSE.txt')]

In [10]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table
     

In [13]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")

wandb: Currently logged in as: shubha_manikarnike. Use `wandb login --relogin` to force relogin


In [14]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (C:\Users\ShubhaManikarnike\.fastai\data\bdd_simple_1k\images)... Done. 6.8s
wandb: Adding directory to artifact (C:\Users\ShubhaManikarnike\.fastai\data\bdd_simple_1k\labels)... Done. 6.9s


In [18]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [20]:
table = _create_table(image_files, BDD_CLASSES)
raw_data_at.add(table, "eda_table")

run.log_artifact(raw_data_at)
run.finish()